# 実行環境の作成

## Google Drive

Google Driveをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## GCP認証

下記コードでGCPに接続

In [2]:
from google.colab import auth
auth.authenticate_user()

# 地域設定

## 関数

In [3]:
import json

def get_area(params={}):
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    areas = json.load(j)

    if not params:
      return areas

    else:
      for p in params.keys():
        areas = list(filter(lambda x: x[p] == params[p], areas))
      return areas

## 都道府県コード一覧

In [4]:
def prefCodes():
  params = {
      'governmentType': 'prefecture',
  }
  
  area = get_area(params)
  return [d.get('areaCode') for d in area]

In [ ]:
print(prefCodes())

['01000', '02000', '03000', '04000', '05000', '06000', '07000', '08000', '09000', '10000', '11000', '12000', '13000', '14000', '15000', '16000', '17000', '18000', '19000', '20000', '21000', '22000', '23000', '24000', '25000', '26000', '27000', '28000', '29000', '30000', '31000', '32000', '33000', '34000', '35000', '36000', '37000', '38000', '39000', '40000', '41000', '42000', '43000', '44000', '45000', '46000', '47000']


## 市区町村コード一覧

政令指定都市の区は無視する

In [164]:
def cityCodes(prefCode=None):
  params = {'governmentType': 'city'}

  if prefCode is not None:
    params['prefCode'] = prefCode

  area = get_area(params)
  area = list(filter(lambda item : item['bigCityFlag'] != '1', area))
  return [d.get('areaCode') for d in area]

In [ ]:
print(cityCodes())

In [ ]:
print(cityCodes(28))

# GBQからデータ取得

## テーブル一覧

In [210]:
from google.cloud import bigquery

def estat_origin_tables():
  project_id='primal-buttress-342908'
  dataset_id = 'estat_origin'
  client = bigquery.Client(project=project_id)
  
  res = []
  for table in client.list_tables(dataset=dataset_id):
    res.append(table.table_id)
  
  return res

In [209]:
print(estat_origin_tables())

['0000010101_A1101', '0000010101_A110101', '0000010101_A110102', '0000010101_A1102', '0000010101_A110201', '0000010101_A110202', '0000010101_A1201', '0000010101_A120101', '0000010101_A120102', '0000010101_A1202', '0000010101_A120201', '0000010101_A120202', '0000010101_A1203', '0000010101_A120301', '0000010101_A120302', '0000010101_A1204', '0000010101_A120401', '0000010101_A120402', '0000010101_A1205', '0000010101_A120501', '0000010101_A120502', '0000010101_A1206', '0000010101_A120601', '0000010101_A120602', '0000010101_A1207', '0000010101_A120701', '0000010101_A120702', '0000010101_A1208', '0000010101_A120801', '0000010101_A120802', '0000010101_A1209', '0000010101_A120901', '0000010101_A120902', '0000010101_A1210', '0000010101_A121001', '0000010101_A121002', '0000010101_A1211', '0000010101_A121101', '0000010101_A121102', '0000010101_A1212', '0000010101_A121201', '0000010101_A121202', '0000010101_A1213', '0000010101_A121301', '0000010101_A121302', '0000010101_A1214', '0000010101_A121401

## テーブルのデータ取得

In [211]:
from google.cloud import bigquery

def get_estat_origin(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_origin'
  client = bigquery.Client(project=project_id)

  query_string = f'SELECT * FROM {dataset_id}.{table_id}'
  
  df = client.query(query_string).result().to_dataframe()

  return df

In [212]:
table_id = '0000010101_A1101'
get_estat_origin(table_id).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName
0,A1101,00000,1975100000,111939643.0,人,総人口,全国,1975年度
1,A1101,00000,1976100000,113094000.0,人,総人口,全国,1976年度
2,A1101,00000,1977100000,114165000.0,人,総人口,全国,1977年度
3,A1101,00000,1978100000,115190000.0,人,総人口,全国,1978年度
4,A1101,00000,1979100000,116155000.0,人,総人口,全国,1979年度


## ランキング付与

In [223]:
import pandas as pd

def df_rank(df):
  
  # 整数値の判定
  values = list(set(df['value'].tolist()))
  isInt = list(map(lambda x: x.is_integer(), values)).count(True) >= len(values)/2
  if isInt:
    df = df.astype({'value': int})
  
  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 都道府県（国内順位）
  df_pref = df[df.copy()['areaCode'].isin(prefCodes())]
  if len(df_pref) != 0:
    df_pref = df_rank_time(df_pref, type='japan')
    df_res = pd.concat([df_res, df_pref])
  
  # 市区町村（国内順位）
  df_city = df[df.copy()['areaCode'].isin(cityCodes())]
  if len(df_city) != 0:
    df_city = df_rank_time(df_city, type='japan')
    df_res = pd.concat([df_res, df_city])

  # 市区町村（県内順位）
  df_city = df[df.copy()['areaCode'].isin(cityCodes())]
  if len(df_city) != 0:
    df_city = df_rank_pref(df_city)
    df_res = pd.merge(df_res, df_city, on=['categoryCode','areaCode','timeCode'], how = 'left')

  return df_res

"""
都道府県ごとに分割してRankを付与する関数

"""
def df_rank_pref(df):
  df_res = pd.DataFrame(index=[], columns=[])

  for p in prefCodes():
    df_city = df[df.copy()['areaCode'].isin(cityCodes(int(p[:2])))]
    df_city = df_rank_time(df_city, type='pref')[['categoryCode','areaCode','timeCode','rankPref']]
    df_res = pd.concat([df_res, df_city])

  return df_res

"""
年次ごとに分割してRankを付与する関数

"""
def df_rank_time(df, type='japan'):
  df_res = pd.DataFrame(index=[], columns=[])

  times = sorted(list(set(df['timeCode'].tolist())))
  for time in times:
      df_time = df.copy().query(f'timeCode == "{time}"').reset_index()
      
      if type == 'japan':
        df_time['rankJapan'] = df_time.rank(ascending=False)['value'].astype(int)
      else:
        df_time['rankPref'] = df_time.rank(ascending=False)['value'].astype(int) 

      df_res = pd.concat([df_res, df_time])

  return df_res


In [224]:
table_id = '0000020201_A1101'
df = get_estat_origin(table_id)
df_rank(df).head()

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan,rankPref
0,0,A1101,01100,1980100000,1401757,人,総人口,北海道 札幌市,1980年度,5,1
1,8,A1101,01202,1980100000,345165,人,総人口,北海道 函館市,1980年度,59,3
2,9,A1101,01203,1980100000,180728,人,総人口,北海道 小樽市,1980年度,133,5
3,10,A1101,01204,1980100000,352619,人,総人口,北海道 旭川市,1980年度,56,2
4,11,A1101,01205,1980100000,150199,人,総人口,北海道 室蘭市,1980年度,169,8


# GBQへ保存

## 単体テスト

In [225]:
from google.cloud import bigquery

def save_gbq(table_id):
  
  # 元データ
  df = get_estat_origin(table_id)
  
  # ランキング付与
  df = df_rank(df)

  # プロジェクト設定
  project_id='primal-buttress-342908'
  dataset_id = 'estat_ranking'
  
  # スキーマ
  job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
  
  # bigqueryへ保存
  client = bigquery.Client(project=project_id)
  job = client.load_table_from_dataframe(df, f'{dataset_id}.{table_id}', job_config=job_config).result()

  # 確認メッセージ  
  table = client.get_table(f'{dataset_id}.{table_id}') 
  print("{} 件のデータを「{}」に保存しました。".format(table.num_rows, table_id))

  return

In [226]:
table_id = '0000020201_A1101'
save_gbq(table_id)

13928 件のデータを「0000020201_A1101」に保存しました。


## 一括処理

In [ ]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# データセットestat-originのテーブル一覧を取得
tables = estat_origin_tables()

# データセットestat-runkingに保存
project_id='primal-buttress-342908'
dataset_id = 'estat_ranking'
client = bigquery.Client(project=project_id)

for table_id in tables:
  try:
    client.get_table(f'{dataset_id}.{table_id}')  
    print("Table {} already exists.".format(table_id))
  except NotFound:
    save_gbq(table_id)

2115 件のデータを「0000010101_A1101」に保存しました。
2115 件のデータを「0000010101_A110101」に保存しました。
2115 件のデータを「0000010101_A110102」に保存しました。
2115 件のデータを「0000010101_A1102」に保存しました。
2115 件のデータを「0000010101_A110201」に保存しました。
2115 件のデータを「0000010101_A110202」に保存しました。
2115 件のデータを「0000010101_A1201」に保存しました。
2115 件のデータを「0000010101_A120101」に保存しました。
2115 件のデータを「0000010101_A120102」に保存しました。
2115 件のデータを「0000010101_A1202」に保存しました。
2115 件のデータを「0000010101_A120201」に保存しました。
2115 件のデータを「0000010101_A120202」に保存しました。
2115 件のデータを「0000010101_A1203」に保存しました。
2115 件のデータを「0000010101_A120301」に保存しました。
2115 件のデータを「0000010101_A120302」に保存しました。
2115 件のデータを「0000010101_A1204」に保存しました。
2115 件のデータを「0000010101_A120401」に保存しました。
2115 件のデータを「0000010101_A120402」に保存しました。
2115 件のデータを「0000010101_A1205」に保存しました。
2115 件のデータを「0000010101_A120501」に保存しました。
2115 件のデータを「0000010101_A120502」に保存しました。
2115 件のデータを「0000010101_A1206」に保存しました。
2115 件のデータを「0000010101_A120601」に保存しました。
2115 件のデータを「0000010101_A120602」に保存しました。
2115 件のデータを「0000010101_A1207」に保存しました。
2115 件のデータを「000001